# MVP

In [1]:
# General Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# AWS Libraries
import logging
import boto3
from botocore.exceptions import ClientError

# NLP Libraries
import unicodedata
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from wordcloud import WordCloud

# Geo-mapping Libraries
import geopandas
import geopy
import folium 

# Helper Functions
import MVP_acquire_ds, MVP_explore

import warnings
warnings.filterwarnings("ignore")

## Data Acquisition

### Download the JSON Files from AWS S3 Bucket

In [2]:
# Create the s3 resource object
s3 = boto3.resource('s3')

# Download the json files from AWS

s3.Bucket("dspreparedjobpostings").download_file("df_ds_tx_prepared_backup.json", 
                                                 "df_ds_tx_prepared_backup.json")

# Print the last modified datetime of the prepared file of data scientist positions in TX
print("Last modified datetime for data scientist positions in TX: ", 
      s3.Object('dspreparedjobpostings', 'df_ds_tx_prepared_backup.json').last_modified)

s3.Bucket("wdpreparedjobpostings").download_file("df_wd_tx_prepared_backup.json", 
                                                 "df_wd_tx_prepared_backup.json")

# Print the last modified datetime of the prepared file of web developer positions in TX
print("Last modified datetime for web developer positions in TX: ", 
      s3.Object('wdpreparedjobpostings', 'df_wd_tx_prepared_backup.json').last_modified)

Last modified datetime for data scientist positions in TX:  2021-02-24 20:46:12+00:00
Last modified datetime for web developer positions in TX:  2021-02-24 20:49:56+00:00


## Data Preparation

### Load JSON Files into Dataframe and Set the Date as the Index

In [3]:
# Read the json file and convert it to pandas dataframe

df_ds = pd.read_json('df_ds_tx_prepared_backup.json')
df_wd = pd.read_json('df_wd_tx_prepared_backup.json')

# Set the date column as the index and sort the index

df_ds.date = pd.to_datetime(df_ds.date)
df_ds = df_ds.set_index('date').sort_index(ascending=False)
df_wd.date = pd.to_datetime(df_wd.date)
df_wd = df_wd.set_index('date').sort_index(ascending=False)

# Print the number of job postings in each dataframe

print("Number of data scientist job postings in TX: ", df_ds.shape[0])
print("Number of web developer job postings in TX: ", df_wd.shape[0])

Number of data scientist job postings in TX:  2265
Number of web developer job postings in TX:  4385


In [4]:
# Print the concise summary of the dataframe df_ds
df_ds.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2265 entries, 2021-02-24 to 2020-12-22
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            2265 non-null   object 
 1   company          2265 non-null   object 
 2   company_rating   2265 non-null   float64
 3   job_link         2265 non-null   object 
 4   job_description  2265 non-null   object 
 5   city             2265 non-null   object 
 6   state            2265 non-null   object 
 7   zipcode          2265 non-null   int64  
 8   clean            2265 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 177.0+ KB


In [5]:
# Print the concise summary of the dataframe df_wd
df_wd.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4385 entries, 2021-02-24 to 2021-01-04
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            4385 non-null   object 
 1   company          4385 non-null   object 
 2   company_rating   4385 non-null   float64
 3   job_link         4385 non-null   object 
 4   job_description  4385 non-null   object 
 5   city             4385 non-null   object 
 6   state            4385 non-null   object 
 7   zipcode          4385 non-null   int64  
 8   clean            4385 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 342.6+ KB


In [6]:
# Print the first 5 rows of the dataframe df_ds
df_ds.head()

,title,company,company_rating,job_link,job_description,city,state,zipcode,clean
date,,,,,,,,,
2021-02-24,Senior Data Analyst - Global Product Group,General Motors,4.1,https://www.indeed.com/rc/clk?jk=e807f01914e80...,About GM\nThere’s never been a more exciting t...,Austin,TX,0,gm there never exciting time work general moto...
2021-02-24,Info Security Sr Engineer - IAM Data Analytics...,Wells Fargo,3.7,https://www.indeed.com/rc/clk?jk=b630f2ae5bc39...,Job Description\nImportant Note: During the ap...,Dallas,TX,0,job description important note application pro...
2021-02-24,Data Solution Architect,Accenture,4.0,https://www.indeed.com/rc/clk?jk=410be44890eef...,"FOR NOW, all Accenture business travel, intern...",Dallas,TX,0,accenture business travel international domest...
2021-02-24,Data Science Intern,Omnitracs LLC,3.4,https://www.indeed.com/rc/clk?jk=926bd9bf7014c...,"Overview:\nOmnitracs, LLC is the source for pr...",Dallas,TX,75201,overview omnitracs llc source private forhire ...
2021-02-24,Business Data Analyst Senior (Remote Work Loca...,USAA,3.9,https://www.indeed.com/rc/clk?jk=46d7c0fe52e9d...,Purpose of Job\nWe are currently seeking a tal...,San Antonio,TX,78288,purpose job currently seeking talented busines...


In [7]:
# Print the first 5 rows of the dataframe df_wd
df_wd.head()

,title,company,company_rating,job_link,job_description,city,state,zipcode,clean
date,,,,,,,,,
2021-02-24,Web Application Developer,ClearCorrect,3.1,https://www.indeed.com/rc/clk?jk=0bdcb1b34c57e...,Summary of Position\nThis position will assist...,Round Rock,TX,78665,summary position position assist developing ma...
2021-02-24,Senior Front-End Developer,"JPMorgan Chase Bank, N.A.",3.9,https://www.indeed.com/rc/clk?jk=16b49f3b45eb9...,Senior Front-End Developer\nAs a Software Engi...,Plano,TX,0,senior frontend developer software engineer be...
2021-02-24,Junior Developers,Omega Project Solutions,0.0,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,We are looking for entry-level developers skil...,Houston,TX,77024,looking entrylevel developer skilled microsoft...
2021-02-24,Front-End Engineer,FlexIT Inc,0.0,https://www.indeed.com/rc/clk?jk=eceabd6e7eab7...,Develop Single Page Applications architecture ...,Dallas,TX,75240,develop single page application architecture d...
2021-02-24,CI/CD Engineer,IBM,3.9,https://www.indeed.com/rc/clk?jk=b27d606451ab4...,Introduction\nSoftware Developers at IBM are t...,Dallas,TX,75202,introduction software developer ibm backbone s...


### Brief Summary of the Job Postings

#### Data Scientist Position

In [8]:
# Which companies hire the most data scientists in TX?
df_ds.company.value_counts().head()

Cognizant Technology Solutions    63
USAA                              53
Facebook                          51
Deloitte                          47
Dell Technologies                 46
Name: company, dtype: int64

In [9]:
# Which cities have the most data scientists postions in TX?
df_ds.city.value_counts().head()

Austin         667
Dallas         372
Houston        283
Plano          182
San Antonio    174
Name: city, dtype: int64

In [10]:
# How does the number of job postings change over time?
df_ds.resample('W').title.count()

date
2020-12-27    392
2021-01-03    136
2021-01-10    212
2021-01-17    187
2021-01-24    352
2021-01-31    294
2021-02-07    259
2021-02-14    268
2021-02-21    118
2021-02-28     47
Freq: W-SUN, Name: title, dtype: int64

In [11]:
# Which company has the best avereage rating? 
df_ds.groupby('company').company_rating.mean().sort_values(ascending=False).head()

company
Digital Pharmacist    5.0
Rekruiters            5.0
Levelset              4.7
Onit                  4.7
Harnham               4.6
Name: company_rating, dtype: float64

In [12]:
df_ds.title.value_counts().head()

Data Scientist               234
Senior Data Scientist         84
Machine Learning Engineer     53
Senior Data Analyst           52
Sr. Data Scientist            38
Name: title, dtype: int64

#### Web Developer Position

In [13]:
# Which companies hire the most web developers in TX?
df_wd.company.value_counts().head()

CyberCoders                            176
Indeed                                 125
JPMorgan Chase Bank, N.A.              125
Infinity Consulting Solutions, Inc.     71
Cognizant Technology Solutions          68
Name: company, dtype: int64

In [14]:
# Which cities have the most web developers postions in TX?
df_wd.city.value_counts().head(4)

Austin     1358
Dallas      611
Houston     491
Plano       349
Name: city, dtype: int64

In [15]:
# How does the number of job postings change over time?
df_wd.resample('W').title.count()

date
2021-01-10      62
2021-01-17     346
2021-01-24     793
2021-01-31    1054
2021-02-07     825
2021-02-14     779
2021-02-21     364
2021-02-28     162
Freq: W-SUN, Name: title, dtype: int64

In [16]:
# Which company has the best avereage rating? 
df_wd.groupby('company').company_rating.mean().sort_values(ascending=False).head()

company
Box                    5.0
The Evolvers Group     5.0
RightNow Ministries    5.0
ALAANT                 5.0
MarketScale            5.0
Name: company_rating, dtype: float64

In [17]:
df_wd.title.value_counts().head()

Software Engineer           95
Web Developer               84
Senior Software Engineer    76
Software Developer          73
Full Stack Developer        63
Name: title, dtype: int64

### Top k Needed Skills

#### Data Scientist Position
- Tech Skills
- Soft Skills
- General Skills (Combination of Tech+Soft)

In [18]:
# Create a tech library
ds_tech_library = ['python','sql','pandas','numpy','matplotlib','scikit learn','scikitlearn','spark','hadoop',
                'aws','amazon web services','azure','microsoft word', 'microsoft excel','excel','tableau',
                'tensor flow','pytorch','hive','impala','matlab','etl','statistics','exploration',
                'extraction','data wrangling','math','machine learning','data visualization','java','js',
                'javascript','scala','r','c','c++','power bi','dashboard','linear algebra','calculus',
                'neural networks','eda','big data','frameworks','database management','testing hypotheses',
                'probability','data mining','perl','nosql','saas','git','github','natural language processing',
                'nlp', 'deep learning','agile','kanban','project management','julia','devops','google cloud',
                'pytorch','computer vision', 'deep neural networks','neural networks','amazon web services',
                'natural language processing','extract,transform,load','mysql','structured query language']

In [19]:
# Print the number of skills in the library
print("Number of data science skills in tech skill library: ", len(ds_tech_library))

# Print the top 5 needed tech skills

ds_top_tech = MVP_explore.top_skills(df_ds, 5, ds_tech_library, 'tech')
ds_top_tech

Number of data science skills in tech skill library:  71
Do you want to save the dataframe as JSON and upload to AWS? (Y/N)
Y
Enter the INITIALS of the job title:
ds


,top5_tech_skills,frequency
0,machine learning,3445.0
1,python,1819.0
2,sql,1396.0
3,aws,1050.0
4,r,1007.0


In [20]:
# Create a soft skill library
ds_soft_library = ['critical thinking','communication','problem solving','teamwork','ethics','business acumen',
                'interpersonal skills','curiosity','storytelling','adaptability','team player','collaboration',
                'time management','leadership','domain knowledge','creativity','decision making',
                'verbal communication','written communication','teamwork']

In [21]:
# Print the number of skills in the library
print("Number of data science skills in soft skill library: ", len(ds_soft_library))

# Print the top 5 needed soft skills

ds_top_soft = MVP_explore.top_skills(df_ds, 5, ds_soft_library, 'soft')
ds_top_soft

Number of data science skills in soft skill library:  20
Do you want to save the dataframe as JSON and upload to AWS? (Y/N)
Y
Enter the INITIALS of the job title:
ds


,top5_soft_skills,frequency
0,communication,1560.0
1,leadership,964.0
2,collaboration,464.0
3,problem solving,346.0
4,written communication,257.0


In [22]:
# Create a general library
ds_general_library = ['python','sql','pandas','numpy','matplotlib','scikit learn','scikitlearn','spark','hadoop',
                'aws','amazon web services','azure','microsoft word', 'microsoft excel','excel','tableau',
                'tensor flow','pytorch','hive','impala','matlab','etl','statistics','exploration',
                'extraction','data wrangling','math','machine learning','data visualization','java','js',
                'javascript','scala','r','c','c++','power bi','dashboard','linear algebra','calculus',
                'neural networks','eda','big data','frameworks','database management','testing hypotheses',
                'probability','data mining','perl','nosql','saas','git','github','natural language processing',
                'nlp', 'deep learning','agile','kanban','project management','julia','devops','google cloud',
                'pytorch','computer vision', 'deep neural networks','neural networks','amazon web services',
                'natural language processing','extract,transform,load','mysql','structured query language'
                'critical thinking','communication','problem solving','teamwork','ethics','business acumen',
                'interpersonal skills','curiosity','storytelling','adaptability','team player','collaboration',
                'time management','leadership','domain knowledge','creativity','decision making',
                'verbal communication','written communication','teamwork']

In [23]:
# Print the number of skills in the library
print("Number of data science skills in general skills library: ", len(ds_general_library))

# Print the top 5 needed general skills

ds_top_general = MVP_explore.top_skills(df_ds, 5, ds_general_library, 'general')
ds_top_general

Number of data science skills in general skills library:  90
Do you want to save the dataframe as JSON and upload to AWS? (Y/N)
Y
Enter the INITIALS of the job title:
ds


,top5_general_skills,frequency
0,machine learning,3445.0
1,python,1819.0
2,communication,1560.0
3,sql,1396.0
4,aws,1050.0


#### Web Developer Position
- Tech skills
- Soft skills
- General Skills (Combination of Tech+Soft)

In [24]:
# Create a tech library
wd_tech_library = ['html','css','javascript','debugging','git','github','libraries','frameworks','publishing web site',
'photoshop','bootstrap','jquery','search engine optimization','seo','basic graphic design','front end','back end',
'visual studio code','visual studio','figma','zeplin','sketch','gatspy','strapi','paint','canva','aws','azure',
'amazon web services','sql','mysql','nosql','node','node.js','js','json','api','google charts','d3','d3.js','react',
'angular','ember','vue','python','c','c++','ruby','ruby on rails','git','github','php','net','.net','java','c#',
'linux','go','gcp','troubleshooting','problem solving','ux','ui','ux/ui','cloud computing','netlify','net']  

In [25]:
# Print the number of skills in the library
print("Number of web dev skills in tech skill library: ", len(wd_tech_library))

# Print the top 5 needed tech skills

wd_top_tech = MVP_explore.top_skills(df_wd, 5, wd_tech_library, 'tech')
wd_top_tech

Number of web dev skills in tech skill library:  67
Do you want to save the dataframe as JSON and upload to AWS? (Y/N)
Y
Enter the INITIALS of the job title:
wd


,top5_tech_skills,frequency
0,javascript,3503.0
1,java,2352.0
2,sql,2123.0
3,react,1891.0
4,net,1696.0


In [26]:
# Create a soft library
wd_soft_library = ['critical thinking','communication','problem solving','teamwork','ethics','business acumen',
                'interpersonal skills','curiosity','storytelling','adaptability','team player','collaboration',
                'time management','leadership','domain knowledge','creativity','decision making',
                'verbal communication','written communication','teamwork'] 

In [27]:
# Print the number of skills in the library
print("Number of web dev skills in soft skill library: ", len(wd_soft_library))

# Print the top 5 needed soft skills

wd_top_tech = MVP_explore.top_skills(df_wd, 5, wd_soft_library, 'soft')
wd_top_tech

Number of web dev skills in soft skill library:  20
Do you want to save the dataframe as JSON and upload to AWS? (Y/N)
n


,top5_soft_skills,frequency
0,communication,2542.0
1,leadership,1010.0
2,collaboration,655.0
3,problem solving,542.0
4,written communication,394.0


In [28]:
# Create a general library
wd_general_library = ['html','css','javascript','debugging','git','github','libraries','frameworks','publishing web site',
'photoshop','bootstrap','jquery','search engine optimization','seo','basic graphic design','front end','back end',
'visual studio code','visual studio','figma','zeplin','sketch','gatspy','strapi','paint','canva','aws','azure',
'amazon web services','sql','mysql','nosql','node','node.js','js','json','api','google charts','d3','d3.js','react',
'angular','ember','vue','python','c','c++','ruby','ruby on rails','git','github','php','net','.net','java','c#',
'linux','go','gcp','troubleshooting','problem solving','ux','ui','ux/ui','cloud computing','netlify','net''critical thinking',
'communication','problem solving','teamwork','ethics','business acumen','interpersonal skills','curiosity',
'storytelling','adaptability','team player','collaboration','time management','leadership','domain knowledge',
'creativity','decision making','verbal communication','written communication','teamwork']

In [29]:
# Print the number of skills in the library
print("Number of web dev skills in general skill library: ", len(wd_general_library))

# Print the top 5 needed general skills

wd_top_tech = MVP_explore.top_skills(df_wd, 5, wd_general_library, 'general')
wd_top_tech

Number of web dev skills in general skill library:  86
Do you want to save the dataframe as JSON and upload to AWS? (Y/N)
N
The dataframe has NOT been saved.


,top5_general_skills,frequency
0,javascript,3503.0
1,communication,2542.0
2,java,2352.0
3,sql,2123.0
4,react,1891.0


### Top 5 Skills Function